#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 25)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,B,WHART,"10,000",10.40,2,DOS,SET,2022-06-01


In [3]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categorise(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,BCH,-3,RD10pct,SET100,3000,18.500000,18.800000,0.000000,0.00%,2,2022-05-10,Buy,55500.000000
5,B,BDMS,-3,1L,SET50,4500,26.250000,27.000000,0.000000,0.00%,2,2022-03-09,Buy,118125.000000
6,B,BGC,-5,DOS,SET,10000,10.000000,10.500000,0.300000,+2.94%,2,2022-05-25,Buy,100000.000000
9,B,IVL,-5,XXX,SET50,2400,48.000000,49.250000,0.500000,+1.03%,2,2022-05-31,Buy,115200.000000
13,S,IP,4,CP1S,mai,2500,20.800000,20.400000,-0.100000,-0.49%,2,2022-02-02,Sell,52000.000000


In [4]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,BCH,-3,RD10pct,SET100,"3,000",18.50,18.80,0.00,0.00%,2,2022-05-10,Buy,"55,500.00"
1,B,ASK,-5,XXX,SET,"3,000",40.00,41.25,-0.75,-1.79%,1,2022-03-03,Buy,"120,000.00"
0,B,AP,-6,XXX,SET100,"10,000",10.00,10.60,-0.20,-1.85%,1,2022-05-10,Buy,"100,000.00"
2,B,BANPU,-7,XXX,SET50,"12,000",11.50,12.20,0.00,0.00%,1,2022-04-11,Buy,"138,000.00"
3,B,BBL,-8,1L,SET50,"2,400",124.00,128.00,1.50,+1.19%,1,2022-04-22,Buy,"297,600.00"


### End of Tables in the process

### Print to verify before upload file

In [14]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [16]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,500",26.00,"117,000.00",1L,SET50,2,2022-03-09
1,B,IVL,"3,000",45.00,"135,000.00",RD05pct,SET50,2,2022-05-31
2,B,WHART,"10,000",10.40,"104,000.00",DOS,SET,2,2022-06-01
3,S,IP,"7,500",20.80,"156,000.00",CP1S,mai,2,2022-02-02


In [17]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    356000.0
S    156000.0
Name: amount, dtype: float64

In [18]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,500",26.00,"117,000.00",1L,SET50,2,2022-03-09
1,B,IVL,"3,000",45.00,"135,000.00",RD05pct,SET50,2,2022-05-31
2,B,WHART,"10,000",10.40,"104,000.00",DOS,SET,2,2022-06-01


In [19]:
cash = 384_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(384000, 356000.0, 28000.0)

In [20]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
3,S,IP,"7,500",20.80,"156,000.00",CP1S,mai,2,2022-02-02


In [21]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(384000, 156000.0, 540000.0)

### After call ord-log (must call everytime that orders change)

In [27]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [28]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [30]:
mb = (df.trans == 'Buy') & (df.spd >= -4)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,BCH,-3,RD10pct,SET100,3000,18.50,18.8,0.00,0.00%,2,2022-05-10,Buy,55500.0
5,B,BDMS,-3,1L,SET50,4200,26.25,27.0,0.00,0.00%,2,2022-03-09,Buy,110250.0
7,B,DIF,-4,DOS,SET,10000,13.50,13.9,-0.10,-0.71%,1,2022-05-17,Buy,135000.0
8,B,HREIT,-4,RD05pct,SET,10000,8.20,8.4,-0.05,-0.59%,1,2022-05-25,Buy,82000.0
9,B,IVL,-4,XXX,SET50,2400,48.00,49.0,0.25,+0.51%,2,2022-05-31,Buy,115200.0


In [31]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [32]:
ms = (df.trans == 'Sell') & (df.spd <= 10)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
13,S,IP,4,CP1S,mai,2500,20.8,20.4,-0.1,-0.49%,2,2022-02-02,Sell,52000.0


In [33]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,BCH,-3,RD10pct,SET100,3000,18.50,18.8,0.00,0.00%,2,2022-05-10,Buy,55500.0
5,B,BDMS,-3,1L,SET50,4200,26.25,27.0,0.00,0.00%,2,2022-03-09,Buy,110250.0
7,B,DIF,-4,DOS,SET,10000,13.50,13.9,-0.10,-0.71%,1,2022-05-17,Buy,135000.0
8,B,HREIT,-4,RD05pct,SET,10000,8.20,8.4,-0.05,-0.59%,1,2022-05-25,Buy,82000.0
9,B,IVL,-4,XXX,SET50,2400,48.00,49.0,0.25,+0.51%,2,2022-05-31,Buy,115200.0
13,S,IP,4,CP1S,mai,2500,20.80,20.4,-0.10,-0.49%,2,2022-02-02,Sell,52000.0


In [34]:
df[mb | ms].shape[0]

6

In [35]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
3,BBL,Buy,"297,600.00",-8
10,KBANK,Buy,"210,000.00",-11
11,KKP,Buy,"204,000.00",-8
2,BANPU,Buy,"138,000.00",-6
7,DIF,Buy,"135,000.00",-4


### Select source of orders between these two

In [36]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
4,Buy,BCH,3000,18.50,2,-3,18.8,0.00,0.00%,RD10pct,SET100,2022-05-10
5,Buy,BDMS,4200,26.25,2,-3,27.0,0.00,0.00%,1L,SET50,2022-03-09
7,Buy,DIF,10000,13.50,1,-4,13.9,-0.10,-0.71%,DOS,SET,2022-05-17
8,Buy,HREIT,10000,8.20,1,-4,8.4,-0.05,-0.59%,RD05pct,SET,2022-05-25
9,Buy,IVL,2400,48.00,2,-4,49.0,0.25,+0.51%,XXX,SET50,2022-05-31
13,Sell,IP,2500,20.80,2,4,20.4,-0.10,-0.49%,CP1S,mai,2022-02-02


In [37]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
4,Buy,BCH,3000,18.50,2,-3,18.8,0.00,0.00%,RD10pct,SET100,2022-05-10
5,Buy,BDMS,4200,26.25,2,-3,27.0,0.00,0.00%,1L,SET50,2022-03-09
6,Buy,BGC,10000,10.00,2,-5,10.5,0.30,+2.94%,DOS,SET,2022-05-25
9,Buy,IVL,2400,48.00,2,-4,49.0,0.25,+0.51%,XXX,SET50,2022-05-31
13,Sell,IP,2500,20.80,2,4,20.4,-0.10,-0.49%,CP1S,mai,2022-02-02


In [38]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process